In [ ]:
import numpy as np
import torch
from scipy.spatial import distance as dist
import numpy as np
import cv2
from PIL import Image 
import time

In [1]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5x, custom

Using cache found in C:\Users\surta/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  8683d4f torch 1.9.0 CUDA:0 (GeForce GTX 1650, 4096.0MB)

Fusing layers... 
C:\Users\surta\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [2]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

In [3]:
def calc_dist_ensemble(image, threshold, breadth, cord_thres, video):    
    # Blue color in BGR 
    color = (255, 0, 0) 

    # Line thickness of 2 px 
    thickness = 2
    conf = 0.3 #confidence threshold for 'person'
    colors = ((0, 0, 255), (240, 0, 159), (0, 165, 255), (255, 255, 0),(255, 0, 255))
    coord_file = cord_thres[labels == 0]
    coord_file = coord_file[coord_file[:,-1] > conf]
    coord_file = coord_file[coord_file[:, 0].argsort()]
    for i in range(len(coord_file)):
        (tl1, tr1, br1, bl1) = (int(coord_file[i][0]), int(coord_file[i][3])), (int(coord_file[i][2]), int(coord_file[i][3])), (int(coord_file[i][2]), int(coord_file[i][1])), (int(coord_file[i][0]), int(coord_file[i][1]))
        for j in range(i+1,len(coord_file)):
            image_copy = image
            (tl2, tr2, br2, bl2) = (int(coord_file[j][0]), int(coord_file[j][3])), (int(coord_file[j][2]), int(coord_file[j][3])), (int(coord_file[j][2]), int(coord_file[j][1])), (int(coord_file[j][0]), int(coord_file[j][1]))
            (xA, yA) = midpoint(tl1, tr1)
            (xB, yB) = midpoint(tl2, tr2)
            mid_pt = midpoint((xA, yA), (xB, yB))
            D = dist.euclidean((xA, yA), (xB, yB))
            D1 = dist.euclidean((xA, yA), mid_pt)
            D2 = dist.euclidean(mid_pt, (xB, yB))
            
            if xB != xA:
                slope = (yB - yA)/(xB - xA)
                if abs(slope) > threshold:
                    width = 5.7 # assumed height of the left most person (in ft.)
                    corrected_D = width * D/(coord_file[i][3] - coord_file[i][1])
                    cv2.putText(image_copy, "{:.1f}ft.".format(corrected_D), (int(xA), int(yA - 20)),cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)
                else:
                    corrected_D_width = breadth * D/(coord_file[i][2] - coord_file[i][0])
                    cv2.putText(image_copy, "{:.1f}ft.".format(corrected_D_width), (int(xA), int(yA - 20)),cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)


                start_point1 = tl1
                end_point1 = br1 
                start_point2 = tl2
                end_point2 = br2 

                # Using cv2.rectangle() method 
                # Draw a rectangle with blue line borders of thickness of 2 px 
                cv2.rectangle(image_copy, start_point1, end_point1, colors[0], thickness)
                cv2.rectangle(image_copy, start_point2, end_point2, colors[1], thickness)
                cv2.line(image, (int(xA), int(yA)), (int(xB), int(yB)),colors[2], thickness)
                video.write(image_copy)
                #cv2.imshow("Image", image)
                #cv2.waitKey()


In [4]:
video_path = 'video.mp4'
video_output = 'density.mp4'

video = cv2.VideoWriter(video_output, 0, 200, (1920,1080))
cap = cv2.VideoCapture(video_path)
frame_num = 0

time_detection = []
time_calculation = []

while cap.isOpened():
        frame_num += 1
        ret, frame = cap.read()

        if not ret:
            print("end of the video file...")
            break
        start1 = time.time()
        image_bounding = cv2.resize(frame, (1920,1080))
        #image_bounding = frame
        results = model(image_bounding)
        labels, cord_thres = results.xyxy[0][:, -1].cpu().numpy(), results.xyxy[0][:, :-1].cpu().numpy()
        end1 = time.time()
        time_detection.append((end1 - start1))
        
        image = cv2.resize(frame, (1920,1080))
        start2 = time.time()
        calc_dist_ensemble(image, 0.5, 1.8, cord_thres, video)
        end2 = time.time()
        time_calculation.append((end2 - start2))

        #cv2.imshow("image with density detection", frame)
        #cv2.waitKey(1)
#cv2.destroyAllWindows()

end of the video file...


#### Mean time for YOLO to detect Human

In [6]:
np.mean(time_detection)

0.05586927623459787

#### Mean time to calculate distance between Humans

In [7]:
np.mean(time_calculation)

0.000559169054031372

In [9]:
result = [time_detection[i] + time_calculation[i]  for i in range(len(time_detection))]

#### Average time needed for each frame to be processed  by the model

In [10]:
np.mean(result)

0.056428445288629246